In [1]:
# NOTE: In this example. We must set `allow_dangerous_request=True` to enable the OpenAPI Agent to automatically use the Request Tool.
# This can be dangerous for calling unwanted requests. Please make sure your custom OpenAPI spec (yaml) is safe.
ALLOW_DANGEROUS_REQUEST = True

In [2]:
import yaml

In [3]:
from langchain_community.agent_toolkits.openapi.spec import reduce_openapi_spec

In [4]:
# with open("openai_openapi.yaml") as f:
#     raw_openai_api_spec = yaml.load(f, Loader=yaml.Loader)
# openai_api_spec = reduce_openapi_spec(raw_openai_api_spec)

# with open("klarna_openapi.yaml") as f:
#     raw_klarna_api_spec = yaml.load(f, Loader=yaml.Loader)
# klarna_api_spec = reduce_openapi_spec(raw_klarna_api_spec)

# with open("spotify_openapi.yaml", encoding="utf-8") as f:
#     raw_spotify_api_spec = yaml.load(f, Loader=yaml.Loader)
# spotify_api_spec = reduce_openapi_spec(raw_spotify_api_spec)

In [5]:
with open("jsonplaceholder_openapi.yaml") as f:
    raw_jsonplaceholder_api_spec = yaml.load(f, Loader=yaml.Loader)
jsonplaceholder_api_spec = reduce_openapi_spec(raw_jsonplaceholder_api_spec)
jsonplaceholder_api_spec

ReducedOpenAPISpec(servers=[{'url': 'https://jsonplaceholder.typicode.com'}], description='See https://jsonplaceholder.typicode.com/', endpoints=[('GET /posts', 'Returns all posts', {'description': 'Returns all posts', 'responses': {'description': 'Successful response', 'content': {'application/json': {'schema': {'type': 'array', 'items': {'type': 'object', 'required': ['id', 'userId', 'title', 'completed'], 'properties': {'id': {'type': 'integer'}, 'userId': {'type': 'integer'}, 'title': {'type': 'string'}, 'completed': {'type': 'string'}}}}}}}}), ('GET /posts/{id}', 'Returns a post by id', {'description': 'Returns a post by id', 'parameters': [{'name': 'id', 'in': 'path', 'required': True, 'description': 'The user id.', 'schema': {'type': 'integer', 'format': 'int64'}}], 'responses': {'description': 'Successful response', 'content': {'application/json': {'schema': {'type': 'object', 'required': ['id', 'userId', 'title', 'completed'], 'properties': {'id': {'type': 'integer'}, 'userId'

In [6]:
with open("apisguru_openapi.yaml") as f:
    raw_apisguru_api_spec = yaml.load(f, Loader=yaml.Loader)
apisguru_api_spec = reduce_openapi_spec(raw_apisguru_api_spec)
apisguru_api_spec

ReducedOpenAPISpec(servers=[{'url': 'https://api.apis.guru/v2'}], description='Wikipedia for Web APIs. Repository of API definitions in OpenAPI format.\n**Warning**: If you want to be notified about changes in advance please join our [Slack channel](https://join.slack.com/t/mermade/shared_invite/zt-g78g7xir-MLE_CTCcXCdfJfG3CJe9qA).\nClient sample: [[Demo]](https://apis.guru/simple-ui) [[Repo]](https://github.com/APIs-guru/simple-ui)\n', endpoints=[('GET /providers.json', 'List all the providers in the directory\n', {'description': 'List all the providers in the directory\n', 'responses': {'description': 'OK', 'content': {'application/json': {'schema': {'type': 'object', 'properties': {'data': {'type': 'array', 'items': {'type': 'string', 'minLength': 1}, 'minItems': 1}}}}}}}), ('GET /{provider}.json', 'List all APIs in the directory for a particular providerName\nReturns links to the individual API entry for each API.\n', {'description': 'List all APIs in the directory for a particular

In [7]:
# import spotipy.util as util
from langchain_community.utilities.requests import RequestsWrapper

# def construct_spotify_auth_headers(raw_spec: dict):
#     scopes = list(
#         raw_spec["components"]["securitySchemes"]["oauth_2_0"]["flows"][
#             "authorizationCode"
#         ]["scopes"].keys()
#     )
#     access_token = util.prompt_for_user_token(scope=",".join(scopes))
#     return {"Authorization": f"Bearer {access_token}"}


# # Get API credentials.
# headers = construct_spotify_auth_headers(raw_spotify_api_spec)
requests_wrapper = RequestsWrapper()

In [8]:
endpoints = [
    (route, operation)
    for route, operations in raw_apisguru_api_spec["paths"].items()
    for operation in operations
    if operation in ["get", "post"]
]
len(endpoints)

7

In [9]:
import tiktoken

enc = tiktoken.encoding_for_model("gpt-4o")


def count_tokens(s):
    return len(enc.encode(s))


count_tokens(yaml.dump(raw_apisguru_api_spec))

2800

In [10]:
from langchain.chat_models import init_chat_model
from langchain_community.agent_toolkits.openapi import planner

llm = init_chat_model("azure_openai:gpt-4o", temperature=0.0)

In [20]:
# NOTE: set allow_dangerous_requests manually for security concern https://python.langchain.com/docs/security
spotify_agent = planner.create_openapi_agent(
    apisguru_api_spec,
    requests_wrapper,
    llm,
    allow_dangerous_requests=ALLOW_DANGEROUS_REQUEST,
)
user_query = (
    # "make me a playlist with the first song from kind of blue. call it machine blues."
    # "Give me the body of the post with ID 1"
    "List all the providers in the directory"
)
spotify_agent.invoke(user_query)



> Entering new AgentExecutor chain...
Action: api_planner
Action Input: I need to find the right API call to list all the providers in the directory.
Observation: 1. Evaluate whether the user query can be solved by the API documented: Yes, the query can be solved using the API endpoints provided.

2. Generate a plan of API calls and say what they are doing step by step:
   - GET /providers.json: This API call will list all the providers in the directory.
Thought:I have the plan to list all the providers in the directory using the API call provided. Now, I will execute this plan to retrieve the information.

Action: api_controller
Action Input: GET /providers.json to list all the providers in the directory.

> Entering new AgentExecutor chain...
Thought: I need to execute the GET request to list all the providers in the directory using the provided API documentation.
Action: requests_get
Action Input: 
```json
{
  "url": "https://api.apis.guru/v2/providers.json",
  "params": {},
  "ou

{'input': 'List all the providers in the directory',
 'output': 'I encountered a 404 error when attempting to list all providers using the endpoint `/providers.json`. The resource might not be available or the endpoint might be incorrect. Please verify the endpoint or contact the API provider for further assistance.'}

In [12]:
user_query = "give me a song I'd like, make it blues-ey"
spotify_agent.invoke(user_query)



> Entering new AgentExecutor chain...
Action: api_planner
Action Input: I need to find the right API calls to recommend a blues song to the user based on their preferences.
Observation: Sorry, this API's domain is related to listing and retrieving information about providers and APIs, not music recommendations or user preferences.
Thought:Thought: The API does not support music recommendations or user preferences related to songs. Therefore, I am unable to assist with this query using the available API tools.
Final Answer: The API does not support music recommendations or user preferences related to songs. I am unable to assist with this query using the available API tools.

> Finished chain.


{'input': "give me a song I'd like, make it blues-ey",
 'output': 'The API does not support music recommendations or user preferences related to songs. I am unable to assist with this query using the available API tools.'}

In [13]:
# headers = {"Authorization": f"Bearer {os.getenv('OPENAI_API_KEY')}"}
# openai_requests_wrapper = RequestsWrapper(headers=headers)

In [14]:
# Meta!
# llm = init_chat_model("azure_openai:gpt-4o", temperature=0.25)
# openai_agent = planner.create_openapi_agent(
#     openai_api_spec, openai_requests_wrapper, llm
# )
# user_query = "generate a short piece of advice"
# openai_agent.invoke(user_query)

In [15]:
from langchain.chat_models import init_chat_model
from langchain_community.agent_toolkits import OpenAPIToolkit, create_openapi_agent
from langchain_community.tools.json.tool import JsonSpec

In [16]:
# with open("openai_openapi.yaml") as f:
#     data = yaml.load(f, Loader=yaml.FullLoader)
json_spec = JsonSpec(dict_=raw_apisguru_api_spec, max_value_length=4000)
json_spec

JsonSpec(dict_={'openapi': '3.0.0', 'x-optic-url': 'https://app.useoptic.com/organizations/febf8ac6-ee67-4565-b45a-5c85a469dca7/apis/_0fKWqUvhs9ssYNkq1k-c', 'x-optic-standard': '@febf8ac6-ee67-4565-b45a-5c85a469dca7/Fz6KU3_wMIO5iJ6_VUZ30', 'info': {'version': '2.2.0', 'title': 'APIs.guru', 'description': 'Wikipedia for Web APIs. Repository of API definitions in OpenAPI format.\n**Warning**: If you want to be notified about changes in advance please join our [Slack channel](https://join.slack.com/t/mermade/shared_invite/zt-g78g7xir-MLE_CTCcXCdfJfG3CJe9qA).\nClient sample: [[Demo]](https://apis.guru/simple-ui) [[Repo]](https://github.com/APIs-guru/simple-ui)\n', 'contact': {'name': 'APIs.guru', 'url': 'https://APIs.guru', 'email': 'mike.ralphson@gmail.com'}, 'license': {'name': 'CC0 1.0', 'url': 'https://github.com/APIs-guru/openapi-directory#licenses'}, 'x-logo': {'url': 'https://apis.guru/branding/logo_vertical.svg'}}, 'externalDocs': {'url': 'https://github.com/APIs-guru/openapi-direc

In [17]:
openapi_toolkit = OpenAPIToolkit.from_llm(
    init_chat_model("azure_openai:gpt-4o", temperature=0),
    json_spec,
    requests_wrapper,
    allow_dangerous_requests=ALLOW_DANGEROUS_REQUEST,
    verbose=True,
)
openapi_toolkit

OpenAPIToolkit(json_agent=AgentExecutor(verbose=True, agent=ZeroShotAgent(llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={}, template='You are an agent designed to interact with JSON.\nYour goal is to return a final answer by interacting with the JSON.\nYou have access to the following tools which help you learn more about the JSON you are interacting with.\nOnly use the below tools. Only use the information returned by the below tools to construct your final answer.\nDo not make up any information that is not contained in the JSON.\nYour input to the tools should be in the form of `data["key"][0]` where `data` is the JSON blob you are interacting with, and the syntax used is Python. \nYou should only use keys that you know for a fact exist. You must validate that a key exists by seeing it previously when calling `json_spec_list_keys`. \nIf you have not seen a key in one of those responses, you ca

In [18]:
openapi_agent_executor = create_openapi_agent(
    llm=init_chat_model("azure_openai:gpt-4o", temperature=0),
    toolkit=openapi_toolkit,
    allow_dangerous_requests=ALLOW_DANGEROUS_REQUEST,
    verbose=True,
)
openapi_agent_executor

AgentExecutor(verbose=True, agent=ZeroShotAgent(llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={}, template='You are an agent designed to answer questions by making web requests to an API given the openapi spec.\n\nIf the question does not seem related to the API, return I don\'t know. Do not make up an answer.\nOnly use information provided by the tools to construct your response.\n\nFirst, find the base URL needed to make the request.\n\nSecond, find the relevant paths needed to answer the question. Take note that, sometimes, you might need to make more than one request to more than one path to answer the question.\n\nThird, find the required parameters needed to make the request. For GET requests, these are usually URL parameters and for POST requests, these are request body parameters.\n\nFourth, make the requests needed to answer the question. Ensure that you are sending the correct parameter

In [19]:
openapi_agent_executor.run(
    # "Make a post request to openai /completions. The prompt should be 'tell me a joke.'"
    # "Give me the body of the post with ID 1"
    "List all the providers in the directory"
)

C:\Users\Admin\AppData\Local\Temp\ipykernel_30896\75989045.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  openapi_agent_executor.run(




> Entering new AgentExecutor chain...
Action: json_explorer
Action Input: "What is the base server URL for the API?"

> Entering new AgentExecutor chain...
Action: json_spec_list_keys
Action Input: data
Observation: ['openapi', 'x-optic-url', 'x-optic-standard', 'info', 'externalDocs', 'servers', 'security', 'tags', 'paths', 'components']
Thought:I see a key named "servers" which might contain information about the server URLs for the API. I should explore this key further to find the base server URL.
Action: json_spec_list_keys
Action Input: data["servers"]
Observation: ValueError('Value at path `data["servers"]` is not a dict, get the value directly.')
Thought:The value at `data["servers"]` is not a dictionary, so I should get the value directly to find the base server URL.

Action: json_spec_get_value
Action Input: data["servers"]
Observation: [{'url': 'https://api.apis.guru/v2'}]
Thought:I now know the final answer.

Final Answer: The base server URL for the API is "https://api.a

'The providers in the directory are:\n1. 1forge.com\n2. 1password.com\n3. 1password.local\n4. 6-dot-authentiqio.appspot.com\n5. ably.io\n6. ably.net\n7. abstractapi.com\n8. adafruit.com\n9. adobe.com\n10. adyen.com\n11. afterbanks.com\n12. agco-ats.com\n13. aiception.com\n14. airbyte.local\n15. airport-web.appspot.com\n16. akeneo.com\n17. alertersystem.com\n18. amadeus.com\n19. amazonaws.com\n20. amentum.space\n21. anchore.io\n22. apache.org\n23. apacta.com\n24. api.ebay.com\n25. api.gov.uk\n26. api.video\n27. api2cart.com\n28. api2pdf.com\n29. apicurio.local\n30. apidapp.com\n31. apideck.com\n32. apigee.local\n33. apigee.net\n34. apimatic.io\n35. apis.guru\n36. apisetu.gov.in\n37. apispot.io\n38. apiz.ebay.com\n39. appcenter.ms\n40. apple.com\n41. apptigent.com\n42. appveyor.com\n43. appwrite.io\n44. archive.org\n45. arespass.net\n46. art19.com\n47. asana.com\n48. asuarez.dev\n49. atlassian.com\n50. ato.gov.au\n51. aucklandmuseum.com\n52. authentiq.io\n53. autodealerdata.com\n54. auto